# Title 
## 2. Data import and cleaning

* Data import and basic cleaning
    * Hate Map Data from SPLC
    * Table 13 from FBI Hate Crime Data
    * Table 11 from FBI Hate Crime Data
* Combining tables


### Hate Map Data from SPLC

The Hate Map data is all from the [SPLC](https://www.splcenter.org/hate-map). They have data from, I believe, 2000 until 2017 available. It shows the names of hate groups, their hate type, and their locations across the united states.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# first, getting the excel file which has each year's data in a different sheet

xl_file = "/Users/gemma/Documents/data science/splc-hate-groups-previous-years.xls"

# reading it in as a dict to then make the dataframe from that dict
sheets_dict = pd.read_excel(xl_file, sheet_name=None)

hatemaps = pd.DataFrame()
for name, sheet in sheets_dict.items():
    sheet['Year'] = int(name)
    # some of the col names arent standardized the same across years - correcting that here
    if 'City/Region' in sheet.columns.values:
        sheet.rename(columns={'City/Region':'City'}, inplace=True)
    if 'Group name' in sheet.columns.values:
        sheet.rename(columns={'Group name':'Group Name'}, inplace=True) 
       
    hatemaps = hatemaps.append(sheet, sort=False)

hatemaps.reset_index(inplace=True, drop=True)

In [3]:
hatemaps.head()

,Group Name,City,State,Hate Type,Year,Unnamed: 4,Unnamed: 5
0,American Christian Dixie Knights of the Ku Klu...,NaN,AL,Ku Klux Klan,2017,NaN,NaN
1,American Confederate Knights of the Ku Klux Klan,NaN,TN,Ku Klux Klan,2017,NaN,NaN
2,Church of the American Christian Knights,Moselle,MS,Ku Klux Klan,2017,NaN,NaN
3,Church of the National Knights of the Ku Klux ...,NaN,KY,Ku Klux Klan,2017,NaN,NaN
4,Church of the National Knights of the Ku Klux ...,Panama,NY,Ku Klux Klan,2017,NaN,NaN


In [4]:
# let's look at the last two columns - 5 contains only NaN, what about 4? 
hatemaps.sort_values(by='Unnamed: 4', ascending=False)

hatemaps.groupby(['Year'])['Unnamed: 4'].count().head(10)

Year
2000     0
2001     0
2002     0
2003     0
2004     0
2005     0
2006     0
2007    90
2008     0
2009     0
Name: Unnamed: 4, dtype: int64

In [5]:
# since columns "Unnamed 4" and "5" contain either no data (5) or data only for 2007 (4)...
hatemaps.drop(['Unnamed: 5', 'Unnamed: 4'], axis=1, inplace=True)

In [6]:
hatemaps = hatemaps.query('(Year > 2003) & (Year < 2017)')

# double checking it worked
hatemaps['Year'].unique()

array([2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006,
       2005, 2004])

In [7]:
# looking at some of the string data that could be inconsistent...

hatemaps['Hate Type'].unique()

array([' Ku Klux Klan', 'Neo-Nazi', 'White Nationalist',
       'Racist Skinhead', 'Christian Identity', 'Black Nationalist',
       'Neo-Confederate', 'Anti-LGBT', 'Anti-Muslim', 'Anti-Immigration',
       'General Hate', 'Holocaust Denial', 'Racist Music',
       'Radical Traditional Catholicism', 'Anti-Immigrant',
       'Ku Klux Klan', 'GH_Anti-Immigration', 'GH_Anti-LGBT',
       'GH_Anti-Muslim', 'GH_Holocaust_Denial', 'GH_Other',
       'GH_Racial_Music', 'GH_Radical_Trad_Catholilicism', 'ANTI-LGBT',
       'Other', 'Radical Traditional Catholilic', 'GH_Anti-Gay',
       'Anti-Gay', 'Radical Traditionalist Catholic', 'General/Anti-Gay',
       'General/Anti-Immigrant', 'General/Holocaust Denial',
       'General/Other', 'General/Racist Music',
       'General/Radical Catholic'], dtype=object)

In [8]:
# preparing a dict of the Hate Group types that are not consistent

corrects = {' Ku Klux Klan': 'Ku Klux Klan','GH_Anti-Muslim' :'Anti-Muslim' , }

corrects.update(corrects.fromkeys(['GH_Anti-LGBT',
                                  'ANTI-LGBT','Anti-Gay','GH_Anti-Gay',
                                  'General/Anti-Gay'],'Anti-LGBT'))

corrects.update(corrects.fromkeys(['GH_Radical_Trad_Catholilicism', 'Radical Traditional Catholicism ',
                                  'Radical Traditional Catholilic',
                                 'Radical Traditionalist Catholic', 
                              'General/Radical Catholic'],'Radical Traditional Catholicism'))

corrects.update(corrects.fromkeys(['Anti-Immigration','GH_Anti-Immigration', 
                               'General/Anti-Immigrant'],'Anti-Immigrant'))

corrects.update(corrects.fromkeys(['General Hate','GH_Other', 
                                'Other'],'General/Other'))

corrects.update(corrects.fromkeys(['GH_Holocaust_Denial', 
                               'General/Holocaust Denial'],'Holocaust Denial'))

corrects.update(corrects.fromkeys(['General/Racist Music','Racist Music', 
                                'GH_Racial_Music', 'Hate Music'],'Racist / Hate Music'))

# a little function
def check_and_replace(value):
    if value in corrects:
        return corrects[value]
    else:
        return value

In [9]:
# making the hate types consistent

hatemaps['Hate Type'] = hatemaps['Hate Type'].apply(check_and_replace)

In [10]:
# the names of the hate groups are also a mess... let's clean them up

# dropping punctuation / strange characters
import re
def cleanup(text):
    # get rid of anything that is not a letter
    text = re.sub(r'[ˆ0-9]+[^\s]+[\W]+\*+','',text)   
    text = re.sub(r'\*+', '', text) # get rid of *s
    text = re.sub(r'[\\x]+', '', text) # get rid of \xs
    text = re.sub(r'\\+', '', text) # get rid of \s
    #text = re.sub(r'"', '', text) # get rid of "s
    text = ' '.join(text.split('  ')) # get rid of extra spaces
    text = text.lower()     # make it all lowercase
    return text

In [11]:
hatemaps['Group Name'] = hatemaps['Group Name'].apply(cleanup)

### Hate Crime Data from FBI 

FBI data from [UCR](https://www.fbi.gov/services/cjis/ucr/publications#Hate-Crime%20Statistics). (Years prior to 2004 are available as pdfs only.)

While there are 14 different tables with information for each year, I'm going to focus on the following two tables from each year:

* Table 11: Offenses: Offense Type by Participating State
* Table 13: Hate Crime Incidents, by Bias Motivation, by State, Agency Type, and Number of Quarters Reported

Year 2012 also has a Table 13 Addendum, which has data that was not included in the rest of the tables because it was submit late to the FBI by the agencies.

Also, Table 13 from 2004 and 2005 is in a different format. So we will narrow down our data to be from 2006 through 2016, unless, from the results, it looks like we will need more data to come to a conclusion..

First, some definitions and functions that I'll need to get and clean the dataframes from the files.

In [12]:
target_years = [x for x in range(2006,2017)] 
folder_names = []

# making a list of all the folders
for year in target_years:
    folder_names.append('Hate Crime Statistics '+ str(year) + ' Tables/')

path = "/Users/gemma/Documents/data science/Hate Crime FBI Data/"


In [13]:
# these are the file names i'll be working with
file_names = ['table4.xls','table11.xls', 'table13.xlsx']

This dictionary of the US States and their abbreviations was provided by [github user rogerallen](https://gist.github.com/rogerallen/1583593).

In [14]:
# going to use this also

us_state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK','Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO','Connecticut': 'CT','Delaware': 'DE',
    'District Of Columbia': 'DC','District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA','Hawaii': 'HI',
    'Idaho': 'ID', 'Illinois': 'IL','Indiana': 'IN','Iowa': 'IA','Kansas': 'KS',
    'Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME','Maryland': 'MD',
    'Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS',
    'Missouri': 'MO','Montana': 'MT', 'Nebraska': 'NE','Nevada': 'NV',
    'New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM','New York': 'NY',
    'North Carolina': 'NC','North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Outlying Areas': 'Other', 'Pennsylvania': 'PA',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN','Texas': 'TX','Utah': 'UT','Vermont': 'VT', 'Virginia': 'VA',
    'Washington': 'WA','West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY',
}

In [15]:
# rename duplicate columns
class renamer():
    def __init__(self):
        self.d = dict()
        
    def __call__(self, x):
        if x not in self.d:
            self.d[x] = 0
            return x
        else:
            self.d[x] += 1
            return "%s_%d" % (x, self.d[x])

First, Table 11.

In [16]:
# to rename the columns to clearer /consistent names
table_11_dict = {'index': 'State_name',
                 'Crimes\nagainst\nsociety1Unnamed: 14_level_1' : 'Crimes c society',
                 'Crimes\nagainst\nsociety3Unnamed: 15_level_1': 'Crimes c society',
                 'Crimes against personsOther1': 'Other Crimes c persons',
                 'Crimes against personsOther3': 'Other Crimes c persons',
                 'Crimes against personsForcible\nrape': 'Rape (legacy)',
                 'Crimes against personsRape\n(legacy\ndefinition)2': 'Rape (legacy)',
                 'Crimes against personsRape\n(legacy\ndefinition2)': 'Rape (legacy)',
                 'Crimes against personsRape\n(revised\ndefinition)1': 'Rape (revised)',
                 'Crimes against personsRape\n(revised\ndefinition1)':'Rape (revised)',
                 'Crimes against propertyOther1': 'Other Crimes c property',
                 'Crimes against propertyOther3': 'Other Crimes c property',
                 'Total\noffensesUnnamed: 0_level_1': 'Total offenses',
                }

In [17]:
def tables_by_year(this_path, header_text, footer_text, year_number):

    # read in the file first to get some info out of it
    onbir = pd.read_excel(this_path)

    # find the start row using the passed in header_text
    found = False
    for col in range(onbir.shape[1]):
        for row in range(onbir.shape[0]):
                if onbir.iat[row,col] == header_text and found == False:
                    row_start = row +1
                    found = True
        # find the last row of content
                if (onbir.iat[row,0] ==  footer_text):
                    last_row = row+1
                    break

    # rereading the file, with the header
    onbir = pd.read_excel(this_path, header=[row_start-1, row_start], 
                          skipfooter=(onbir.shape[0]-last_row))
    
    
    # right now it has multilvel columns. flattening them to 1 column and keeping the names
    colnames = onbir.columns
    ind = pd.Index([e[0] + e[1] for e in colnames.tolist()])
    onbir.columns = ind
    
    # moving the index over
    onbir.reset_index(inplace=True)
    
    #fixing the names
    onbir.rename(index=str, columns=table_11_dict, inplace=True)
    
    # add column for the year
    onbir['Year'] = target_years[year_number]
    
    #adding state abbrev
    onbir['State'] = onbir['State_name'].map(us_state_abbrev)
    
    return onbir

In [18]:
# function to put all of the table 11s into one dataframe

def combine_tables(list_of_paths, header_text, footer_text):
    
    df_dict = {}
        
    for i, name in enumerate(list_of_paths):
        df_dict[name] = tables_by_year(name, header_text,footer_text, i)
        
    table_11 = pd.concat(df_dict.values(), axis=0, ignore_index=True, sort=True)
    
    return table_11

In [19]:
# after reviewing the files i found these delimited the header and footers

header_text = 'Intimidation'
footer_text = 'Wyoming'

list_of_paths_11 = [ str(path+folder+file_names[1]) for folder in folder_names]

crimes_by_state = combine_tables(list_of_paths_11, header_text, footer_text)

In [20]:
crimes_by_state.head()

,Crimes against personsAggravated assault,Crimes against personsIntimidation,Crimes against personsMurder and nonnegligent manslaughter,Crimes against personsSimple assault,Crimes against propertyArson,Crimes against propertyBurglary,Crimes against propertyDestruction/ damage/ vandalism,Crimes against propertyLarceny- theft,Crimes against propertyMotor vehicle theft,Crimes against propertyRobbery,Crimes c society,Other Crimes c persons,Other Crimes c property,Rape (legacy),Rape (revised),State,State_name,Total offenses,Year
0,1178,2508,3,1737,41,155,2911,261,25,142,38,17,58,6.0,NaN,NaN,Total,9080,2006
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,NaN,AL,Alabama,1,2006
2,3,2,0,5,0,0,1,0,0,0,0,0,0,0.0,NaN,AK,Alaska,11,2006
3,23,95,0,40,0,2,54,0,1,0,0,0,0,0.0,NaN,AZ,Arizona,215,2006
4,16,27,0,36,0,8,16,23,0,1,0,1,4,1.0,NaN,AR,Arkansas,133,2006


Next, Table 13.

In [21]:
def make_table_13s(this_path, number):
       
    # first time, to find where the header ends
    full_table = pd.read_excel(this_path, index_col=0)
    
    # then we're going to find which row Religion is in, bc it's one category
    # of bias that is in all of the excel files
    
    found = False
    for col in range(full_table.shape[1]):
        for row in range(full_table.shape[0]):
            if full_table.iat[row,col] == "Religion" and found == False:
                row_start = row + 1
                found = True
    
    if found == False:
        print("honey you have a storm coming")
        
    # now we're legit reading it in, with the state as the index
    full_table = pd.read_excel(this_path, header=row_start) #index_col=0)

    # the size of the footer is diff each table, but all of the values are Na, so we'll 
    # get rid of the footer through dropping rows that have only Na values
    full_table.dropna(axis=0, how="all", inplace=True)
    
    # renaming columns that are not consistently named across all files, or just bc
    full_table.rename(index=str, 
                      columns={ 'Agency Type': 'Agency type', 
                               'Gender\nIdentity':'Gender Identity',
                               'Race/\nEthnicity/\nAncestry': 'REA'},
                      inplace=True)

    # there's one more inconsistency, it has to do with changes in how the 
    # biases were classified - they used to split up race & ethnicity, 
    # then starting from 2015 they lumped together w/ ancestry
    
    if 'Race' in full_table.columns.values:
        full_table['REA'] = (full_table['Race'] + full_table['Ethnicity'])  
    
    # filling in the missing data in these 3 columns based on looking at the excel file
    full_table['State'] = pd.Series(full_table.State).fillna(method="ffill")
    full_table['Agency type'] = pd.Series(full_table['Agency type']).fillna(method="ffill")
    full_table['Agency name'] = full_table['Agency name'].fillna("Total")
    
    # adding the year for organization when the dfs are all merged
    full_table['Year'] = target_years[number] # needs 2b same as # folder_names & target_years
    
    return full_table

In [22]:
# function to put all of the table 13s into one dataframe

def combine_table13s(list_of_paths):
    
    df_dict = {}
        
    for i, name in enumerate(list_of_paths):
        df_dict[name] = make_table_13s(name, i)
        
    table_13 = pd.concat(df_dict.values(), axis=0, ignore_index=True, sort=True)
    
    # drop cols i don't plan on needing 
    table_13.drop(['1st\nquarter', '2nd\nquarter', '3rd\nquarter', '4th\nquarter', 
                  'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15','Unnamed: 16', 
                   'Unnamed: 17', 'Unnamed: 3'], axis=1, inplace=True)
    
    # reorder the cols for convenience
    table_13 = table_13[['Year','State','Agency type','Agency name',
                         'REA', 
                         'Religion', 'Disability', 'Gender','Gender Identity',
                         'Sexual\norientation','Race','Ethnicity','Population']]
    
    # fix capitalization w helper function
    for col in ['State', 'Agency type', 'Agency name']:
        table_13[col] = table_13[col].apply(str.title)
        
    table_13.rename(index=str,columns={'State': 'State_name', 
                                       'Sexual\norientation': 'Sexual orientation'}, inplace=True)
       
    # drop anything that isn't a state - my above method of removing the footer didnt work 
    return table_13.loc[table_13['State_name'].isin(us_state_abbrev.keys())]

In [23]:
list_of_paths = [ str(path+folder+file_names[2]) for folder in folder_names]

bias_by_state = combine_table13s(list_of_paths)

In [24]:
# dropping redundant columns or ones that I won't use
partial_bbs = bias_by_state.drop(['Race', 'Ethnicity','Population'], axis=1, inplace=False)

In [25]:
partial_bbs['State'] = partial_bbs['State_name'].map(us_state_abbrev)
partial_bbs.head()

,Year,State_name,Agency type,Agency name,REA,Religion,Disability,Gender,Gender Identity,Sexual orientation,State
0,2006,Alabama,Total,Total,1.0,0,0.0,NaN,NaN,0.0,AL
1,2006,Alabama,Cities,Total,1.0,0,0.0,NaN,NaN,0.0,AL
2,2006,Alabama,Cities,Atmore,1.0,0,0.0,NaN,NaN,0.0,AL
3,2006,Alaska,Total,Total,4.0,0,0.0,NaN,NaN,2.0,AK
4,2006,Alaska,Cities,Total,4.0,0,0.0,NaN,NaN,2.0,AK


Combining all the tables. Since we have data that applies broadly (state level) and data by city, I'll make two separate tables that we can use to analyze at two different scopes.

In [26]:
# making two separate tables so as to not duplicate the info

state_totals = partial_bbs.loc[(partial_bbs['Agency name'] == 'Total') & (partial_bbs['Agency type'] == 'Total')]
state_totals = state_totals[['Year','State', 'REA', 'Religion',
       'Disability', 'Gender', 'Gender Identity', 'Sexual orientation']]

by_city_only = partial_bbs.loc[(partial_bbs['Agency name'] != 'Total') & (partial_bbs['Agency type'] != 'Total')]


In [27]:
by_city_only = pd.merge(by_city_only.drop(['Agency type'],1), hatemaps, how="outer",  
                        right_on=['Year','State','City'], 
         left_on=['Year','State', 'Agency name']).fillna(0)

In [28]:
#by_city_only.rename(index=str, columns={'Race / Ethnicity / Ancestry':'REA'}, inplace=True)

Below when I was trying to work with this dataframe, I found a ' ' in one of the columns was preventing the model from running.

In [29]:
for col in ['REA', 'Religion','Disability', 'Gender', 'Gender Identity', 
            'Sexual orientation']:
    print(col, set([type(x) for x in by_city_only[col]]))
    
by_city_only['Gender'].unique()

REA {<class 'float'>}
Religion {<class 'float'>}
Disability {<class 'float'>}
Gender {<class 'str'>, <class 'int'>, <class 'float'>}
Gender Identity {<class 'float'>}
Sexual orientation {<class 'float'>}


array([0, 1, 4, ' ', 2, 3.0, 5.0], dtype=object)

In [30]:
by_city_only[by_city_only['Gender'] == ' ']

,Year,State_name,Agency name,REA,Religion,Disability,Gender,Gender Identity,Sexual orientation,State,Group Name,City,Hate Type
16555,2013,Nevada,Reno,2.0,1.0,0.0,,0.0,1.0,NV,0,0,0


In [31]:
by_city_only.iloc[16555, 6] = 0

In [32]:
# export them to csv to use in the rest of the notebooks

#hatemaps.to_csv('/Users/gemma/Documents/data science/fc-hatemaps.csv')
#crimes_by_state.to_csv('/Users/gemma/Documents/data science/fc-crimes_by_state.csv')
#state_totals.to_csv('/Users/gemma/Documents/data science/fc-hate-state_totals.csv')
#by_city_only.to_csv('/Users/gemma/Documents/data science/fc-hate-by_city.csv')


In [33]:
#partial_bbs.to_csv('/Users/gemma/Documents/data science/fc-partial_bias_byst.csv')